# 3D printing a turbulent box

In [ ]:
from pathlib import Path
import sys

import numpy as np
from PIL import Image
from scipy.interpolate import RegularGridInterpolator

import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm

from IPython.display import display, HTML

from tqdm import tqdm

plt.style.use([{'image.cmap':'gray_r'}])

Read data. Data is currently (until 01.02.2022) available [here](https://gigamove.rwth-aachen.de/de/download/4abe80f1c550806021f85af8c57c886e).

In [ ]:
f = np.load('turbulentbox.npy')
data = f.copy()
del f

## Normalization

Find the largest magnitude of the data values and define a logarithmic norm

In [ ]:
vmax = 10**np.ceil(np.log10(data.max()))
norm = LogNorm(1e-2 * vmax, vmax, clip=True)

## Example plot

Select which slice to plot

In [ ]:
i = 0

In [ ]:
# apply the norm

img_norm = np.array(norm(data[:, :, i]))

# we could now make this straight to an image
im = Image.fromarray(np.uint8(255 - img_norm * 255))
display('Image:', im)

# This is all we need to do to use dithering

im_1 = im.convert("1")
display('Dithered image:', im_1)

## Upscale the data

### Coordinates & aspect ratios

these are the original "coordinates" of the pixels

In [ ]:
x = np.arange(data.shape[0])
y = np.arange(data.shape[1])
z = np.arange(data.shape[2])

create an interpolation function for the 3d data

In [ ]:
f_interp = RegularGridInterpolator((x, y, z), data)

settings of the printer (educated guess, especially the layer thickness might be different)

In [ ]:
height = 10 # 10 cm, this should be the total height of the printed cube
layer_thickness = 55e-4 # 14 micron
dpi_x = 600
dpi_y = 300

calculate the new grids in x, y, z

In [ ]:
n_z = int(height / layer_thickness)
dpi_z = int(2.54 // layer_thickness)

n_x = int(n_z / dpi_z * dpi_x)
n_y = int(n_z / dpi_z * dpi_y)

n_y += n_y%2 # add 1 to make it even if it isn't

x2 = np.linspace(0, data.shape[0] - 1, n_x)
y2 = np.linspace(0, data.shape[1] - 1, n_y)
z2 = np.linspace(0, data.shape[2] - 1, n_z)

this creates the coordinates of the new layer and we'll update the `z` coordinate as we go, interpolating one layer at a time. `x` and `y` stay the same

In [ ]:
coords = np.concatenate((np.meshgrid(x2, y2, z2[0])), axis=-1)

### Iteration

we get the new layer by interpolating the 3D data. We store the images in the path set by `output_dir`.

In [ ]:
output_dir = 'slices'

Prepare output folder

In [ ]:
path = Path(output_dir)

if not path.is_dir():
    path.mkdir()
else:
    files = list(path.glob('slice*.png'))
    if len(files)>0:
        print('directory exists, deleting old files')
        for file in files:
            file.unlink()

This function interpolates one layer, creates and dithers the image and writes it to file

In [ ]:
def makeslice(iz, z2, f_interp, coords, norm, path):
    """
    iz : int
        slice index within `z2`
        
    z2 : array
        the new vertical coordinate array
        
    f_inter : callable
        the interpolation function of (x,y,z)
        
    norm : callable
        the normalization function that maps density to 0...1
        
    path : str | Path
        the path into which to store the images
    """
    # update coordinates - only last entry changes
    n_y, n_x = coords.shape[:-1]
    copy = coords.copy()
    copy[:, :, -1] = z2[iz]
    
    # interpolate
    new_layer = f_interp(copy.reshape([-1, 3])).reshape([n_x, n_y]).T
    
    # normalize, convert to grayscale image
    layer_norm = np.array(norm(new_layer))
    im = Image.fromarray(np.uint8(255 - layer_norm * 255)).convert('1')
    
    # save as 1bit bitmap
    im.save(path / f'slice_{iz:04d}.png', bits=1, optimize=True)

This worker function is meant as a wrapper to parallelize the iteration, but somehow this doesn't work yet, so the loop is serial.

In [ ]:
def worker(iz):
    try:
        makeslice(iz, z2, f_interp, coords, norm, path)
        return 0
    except Exception:
        raise Exception

In [ ]:
n = 2 # how many slices to make, all would be len(z2)

In [ ]:
res = list(tqdm(map(worker, range(n)), total=n))

# Optional: Make movies

In [ ]:
from IPython import get_ipython
if get_ipython() is None:
    raise NotImplementedError('you need to run this with ipython to continue here')
    sys.exit(1)

### Movie 1: stitch the dithered slices to a movie

In [ ]:
!ffmpeg -y -i slices/slice_%04d.png -c:v libx264 -crf 15 -maxrate 400k -pix_fmt yuv420p -r 20 -bufsize 1835k movie_01.mp4

In [ ]:
HTML(f"""
<video width="500" controls>
  <source src="movie_01.mp4" type="video/mp4">
</video>
""")

### Movie 2: stick the non-dithered slices to a movie

In [ ]:
# prepare the figure

vmax = 10**np.ceil(np.log10(data.max()))
f, ax = plt.subplots()
ax.set_aspect('equal')
cc = ax.pcolormesh(data[:, :, 0], norm=LogNorm(1e-2 * vmax, vmax))
ax.set_axis_off()

# prepare the output directory

frames_path = Path('frames')
if not frames_path.is_dir():
    frames_path.mkdir()
    
n_frames = data.shape[-1]-1
    
for i in tqdm(range(1, n_frames)):
    cc.set_array(data[:, :, i].T.ravel())
    f.savefig(frames_path / f'frame_{i:04d}.jpg', bbox_inches='tight', dpi=210)
    
print('\r--- DONE! ---'.ljust(20))

In [ ]:
!ffmpeg -y -i {frames_path}/frame_%04d.jpg -c:v libx264 -crf 15 -maxrate 400k -pix_fmt yuv420p -r 20 -bufsize 1835k movie_02.mp4

In [ ]:
HTML(f"""
<video width="500" controls>
  <source src="movie_02.mp4" type="video/mp4">
</video>
""")